In [ ]:
# Move kaggle.json into the ~/.kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set permissions to avoid permission errors
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)
print("Contents of dataset directory:", os.listdir(path))
print("Contents of dataset directory:", os.listdir(os.path.join(path, "chest_xray", "chest_xray")))

# Define a function to count images in a given directory
def count_images_in_subsets(base_dir, subsets=("train", "val", "test")):
    results = {}
    grand_total = 0  # Track total images across all subsets

    for subset in subsets:
        subset_dir = os.path.join(base_dir, subset)
        counts = {}
        total = 0

        if os.path.exists(subset_dir):
            for class_name in os.listdir(subset_dir):
                class_dir = os.path.join(subset_dir, class_name)
                if os.path.isdir(class_dir):
                    num_files = len([
                        f for f in os.listdir(class_dir)
                        if os.path.isfile(os.path.join(class_dir, f))
                    ])
                    counts[class_name] = num_files
                    total += num_files

        results[subset] = {"total": total, "counts": counts}
        grand_total += total

    results["grand_total"] = grand_total
    return results

# Based on the printed structure, the actual images seem to reside in:
base_dir = os.path.join(path, "chest_xray", "chest_xray")

count_images_in_subsets(base_dir)

100%|██████████| 2.29G/2.29G [01:17<00:00, 31.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2
Contents of dataset directory: ['chest_xray']
Contents of dataset directory: ['train', '.DS_Store', 'val', 'test']


{'train': {'total': 5218, 'counts': {'NORMAL': 1342, 'PNEUMONIA': 3876}},
 'val': {'total': 18, 'counts': {'NORMAL': 9, 'PNEUMONIA': 9}},
 'test': {'total': 624, 'counts': {'NORMAL': 234, 'PNEUMONIA': 390}},
 'grand_total': 5860}

In [ ]:
import os
import shutil

# Ensure 'data/' directory exists
os.makedirs('data', exist_ok=True)

# Move all files/directories from 'base_dir' to 'data/'
for item in os.listdir(base_dir):
    shutil.move(os.path.join(base_dir, item), 'data/')

print("Files successfully moved to 'data/' directory.")

count_images_in_subsets('data')

Files successfully moved to 'data/' directory.


{'train': {'total': 5218, 'counts': {'NORMAL': 1342, 'PNEUMONIA': 3876}},
 'val': {'total': 18, 'counts': {'NORMAL': 9, 'PNEUMONIA': 9}},
 'test': {'total': 624, 'counts': {'NORMAL': 234, 'PNEUMONIA': 390}},
 'grand_total': 5860}

Data Loading and Preprocessing

Rebalancing the dataset split:
A very small validation set (only 16 images) may not provide a robust estimate for hyperparameter tuning. 70/20/10 split (train/validation/test) could be  better for training stability and reliable validation feedback.

Class imbalance:
The training data shows a significant imbalance (NORMAL: 1342 vs PNEUMONIA: 3876). We can compute class weights so that the model can give more importance to the minority class.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import os
import random
import shutil

def split_dataset(original_dir, train_dir, val_dir, test_dir,
                  train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    """
    Splits the dataset from the original_dir into train, validation, and test sets.
    original_dir should contain subdirectories for each class with image files.
    """
    random.seed(seed)

    # Ensure target directories exist (or re-create them)
    for d in [train_dir, val_dir, test_dir]:
        if os.path.exists(d):
            shutil.rmtree(d)
        os.makedirs(d)

    # Iterate over each class in the original dataset
    for class_name in os.listdir(original_dir):
        class_dir = os.path.join(original_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Only consider files (ignore directories)
        images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        random.shuffle(images)
        total = len(images)
        train_count = int(total * train_ratio)
        val_count = int(total * val_ratio)

        train_images = images[:train_count]
        val_images = images[train_count:train_count + val_count]
        test_images = images[train_count + val_count:]

        # Create class-specific directories in target folders if they don't exist
        for d in [train_dir, val_dir, test_dir]:
            target_class_dir = os.path.join(d, class_name)
            if not os.path.exists(target_class_dir):
                os.makedirs(target_class_dir)

        # Copy the files to the appropriate directories
        for img in train_images:
            shutil.copy2(os.path.join(class_dir, img), os.path.join(train_dir, class_name, img))
        for img in val_images:
            shutil.copy2(os.path.join(class_dir, img), os.path.join(val_dir, class_name, img))
        for img in test_images:
            shutil.copy2(os.path.join(class_dir, img), os.path.join(test_dir, class_name, img))

def merge_datasets(subset_dirs, merged_dir):
    """
    Merges images from a list of subset directories (e.g. train, val, test)
    into a single directory with the same class subdirectories.
    """
    if os.path.exists(merged_dir):
        shutil.rmtree(merged_dir)
    os.makedirs(merged_dir)

    for subset in subset_dirs:
        subset_dir = os.path.join("data", subset)
        for class_name in os.listdir(subset_dir):
            src_class_dir = os.path.join(subset_dir, class_name)
            if not os.path.isdir(src_class_dir):
                continue
            dst_class_dir = os.path.join(merged_dir, class_name)
            if not os.path.exists(dst_class_dir):
                os.makedirs(dst_class_dir)
            for file in os.listdir(src_class_dir):
                src_file = os.path.join(src_class_dir, file)
                if os.path.isfile(src_file):
                    dst_file = os.path.join(dst_class_dir, file)
                    # If a file with the same name exists, append a random number.
                    if os.path.exists(dst_file):
                        base, ext = os.path.splitext(file)
                        dst_file = os.path.join(dst_class_dir, f"{base}_{random.randint(1000,9999)}{ext}")
                    shutil.copy2(src_file, dst_file)

# Merge the existing train, val, and test folders into one merged directory.
merged_dir = os.path.join("data", "merged")
merge_datasets(subset_dirs=["train", "val", "test"], merged_dir=merged_dir)

# Re-split the merged dataset into new train, val, and test folders (70/20/10)
train_dir = os.path.join("data", "train")
val_dir   = os.path.join("data", "val")
test_dir  = os.path.join("data", "test")
split_dataset(merged_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)

image_counts = count_images_in_subsets('data')
print(image_counts)

# Image augmentation and data generators
IMG_HEIGHT = 180
IMG_WIDTH  = 180

# Create an ImageDataGenerator for augmentation on training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,       # normalise pixel values
    rotation_range=15,       # random rotations up to 15 degrees
    width_shift_range=0.1,   # horizontal shift 10%
    height_shift_range=0.1,  # vertical shift 10%
    shear_range=0.1,         # shear by 10%
    zoom_range=0.1,          # zoom in/out 10%
    horizontal_flip=True,    # flip horizontally
    fill_mode='nearest'      # fill pixels after transform
)
# For validation and test, we only rescale (no augmentation)
val_datagen  = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

image_counts = count_images_in_subsets('data')
print(image_counts)

# Compute class weights to address imbalance in the training data.
class_labels = train_gen.classes
classes = np.unique(class_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)
class_weights = dict(enumerate(class_weights))
print("Computed class weights:", class_weights)

{'train': {'total': 4101, 'counts': {'NORMAL': 1109, 'PNEUMONIA': 2992}}, 'val': {'total': 1172, 'counts': {'NORMAL': 317, 'PNEUMONIA': 855}}, 'test': {'total': 587, 'counts': {'NORMAL': 159, 'PNEUMONIA': 428}}, 'grand_total': 5860}
Found 4099 images belonging to 2 classes.
Found 1171 images belonging to 2 classes.
Found 586 images belonging to 2 classes.
{'train': {'total': 4101, 'counts': {'NORMAL': 1109, 'PNEUMONIA': 2992}}, 'val': {'total': 1172, 'counts': {'NORMAL': 317, 'PNEUMONIA': 855}}, 'test': {'total': 587, 'counts': {'NORMAL': 159, 'PNEUMONIA': 428}}, 'grand_total': 5860}
Computed class weights: {0: 1.8514001806684734, 1: 0.6849933155080213}


Build the CNN model

✅ Added SE Attention Block (at the last convolutional block)

✅ Increased Depth to 5 Convolutional Layers (compared to the original 3)

✅ Used Multi-Scale Feature Extraction (3x3 for details, 5x5 for larger features)

✅ Replaced Flatten with Global Average Pooling (GAP reduces parameters)

✅ Optimized Fully Connected Layers (Dense 256 instead of multiple layers)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization,
    Dropout, GlobalAveragePooling2D, Dense, Input,
    Multiply, Reshape
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define image dimensions and channels
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

# Define a custom SE (Squeeze-and-Excitation) Block as a Keras layer
@tf.keras.utils.register_keras_serializable()
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, reduction=16, **kwargs):
        super(SEBlock, self).__init__(**kwargs)
        self.reduction = reduction

    def build(self, input_shape):
        filters = input_shape[-1]
        self.global_avg_pool = GlobalAveragePooling2D()
        self.reshape = Reshape((1, 1, filters))
        self.dense1 = Dense(filters // self.reduction, activation='relu',
                            kernel_initializer='he_normal', use_bias=False)
        self.dense2 = Dense(filters, activation='sigmoid',
                            kernel_initializer='he_normal', use_bias=False)
        super(SEBlock, self).build(input_shape)

    def call(self, inputs):
        x = self.global_avg_pool(inputs)
        x = self.reshape(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return Multiply()([inputs, x])

    def get_config(self):
        config = super(SEBlock, self).get_config()
        config.update({'reduction': self.reduction})
        return config

# Build the CNN model with the architectural modifications
model = Sequential([
    Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),

    # Conv Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Conv Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Conv Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),  # Reduce overfitting

    # Conv Block 4
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Conv Block 5 - Increased depth with a 5×5 kernel for broader feature extraction
    Conv2D(512, (5, 5), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Incorporate SE Attention Block to recalibrate channel-wise features
    SEBlock(reduction=16),

    # Replace Flatten with Global Average Pooling (GAP)
    GlobalAveragePooling2D(),

    # Improved Fully Connected Layers
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # Output for binary classification
])

# View model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 14, 14, 512)         │       3,277,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 14, 14, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ se_block (SEBlock)                   │ (None, 7, 7, 512)           │          32,7

 Total params: 3,835,073 (14.63 MB)

 Trainable params: 3,832,577 (14.62 MB)

 Non-trainable params: 2,496 (9.75 KB)

Model Training

In [ ]:
# Compile the model with binary_crossentropy and multiple metrics
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn')
    ]
)

# Set callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Train the model
# Ensure train_gen, val_gen, and class_weights are defined prior to training.
history = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    callbacks=[early_stop, lr_reduce, checkpoint],
    class_weight=class_weights
)

Epoch 1/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 908s 7s/step - accuracy: 0.7848 - auc: 0.8864 - fn: 345.3000 - fp: 55.4769 - loss: 0.4858 - precision: 0.9510 - recall: 0.7501 - tn: 490.2538 - tp: 1182.6384 - val_accuracy: 0.7293 - val_auc: 0.5000 - val_fn: 0.0000e+00 - val_fp: 317.0000 - val_loss: 4.6786 - val_precision: 0.7293 - val_recall: 1.0000 - val_tn: 0.0000e+00 - val_tp: 854.0000 - learning_rate: 0.0010
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 883s 7s/step - accuracy: 0.8609 - auc: 0.9410 - fn: 221.0846 - fp: 63.6769 - loss: 0.3182 - precision: 0.9522 - recall: 0.8534 - tn: 493.9769 - tp: 1294.9308 - val_accuracy: 0.7293 - val_auc: 0.5000 - val_fn: 0.0000e+00 - val_fp: 317.0000 - val_loss: 5.3000 - val_precision: 0.7293 - val_recall: 1.0000 - val_tn: 0.0000e+00 - val_tp: 854.0000 - learning_rate: 0.0010
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 892s 7s/step - accuracy: 0.9100 - auc: 0.9679 - fn: 151.1154 - fp: 39.6308 - loss: 0.2291 - precision: 0.9743 - recall: 0.9019 - tn: 512.4385 - tp

Model Evaluation

In [ ]:
# Evaluate the model and get results
print("\nKeras Model Evaluation Metrics:")
results = model.evaluate(
    test_gen,
    verbose=1,
    return_dict=True
)

# Display all metrics with clear formatting
print("\nDetailed Model Evaluation Results:")
print("-" * 40)
for metric_name, value in results.items():
    # Format the metric name to be more readable
    formatted_name = metric_name.replace('_', ' ').title()
    # Handle different numeric formats appropriately
    if isinstance(value, (int, float)):
        if metric_name in ['tp', 'tn', 'fp', 'fn']:  # Count metrics
            print(f"{formatted_name}: {int(value)}")
        else:  # Percentage metrics
            print(f"{formatted_name}: {value:.4f}")


Keras Model Evaluation Metrics:
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.9471 - auc: 0.7944 - fn: 2.3500 - fp: 13.8000 - loss: 0.1585 - precision: 0.6840 - recall: 0.7899 - tn: 129.4000 - tp: 186.6500

Detailed Model Evaluation Results:
----------------------------------------
Accuracy: 0.9625
Auc: 0.9929
Fn: 6
Fp: 16
Loss: 0.1002
Precision: 0.9634
Recall: 0.9859
Tn: 143
Tp: 421
